In [ ]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform
import matplotlib.pyplot as plt

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")


In [ ]:
df_merge= pd.read_csv('../../Data/날씨+일별주차이용.csv')

In [ ]:
pd.read_csv('../../Data/한강2403-2409.csv')
df = pd.read_csv('../../Data/서울시 한강공원 월별 이용객 현황.csv',encoding='euc-kr')


In [ ]:

print(df.columns)
df = df.loc[:,['공원 코드','공원명','일반이용자(아침)','일반이용자(낮)', '일반이용자(저녁)','현황 일시']]

In [ ]:
df['현황 일시']= pd.to_datetime(df['현황 일시'])

In [ ]:
df_merge

In [ ]:
# PLT-001: 강서   
# PLT-002: 난지   
# PLT-003: 망원   
# PLT-004: 이촌   
# PLT-005: 뚝섬       
# PLT-006: 광나루     
# PLT-007: 양화   
# PLT-008: 여의도     
# PLT-009: 반포   
# PLT-0010: 잠원  
# PLT-0011: 잠실
                             
# Hzone011: 강서  
# Hzone009: 난지  
# Hzone010: 망원  
# Hzone006: 이촌  
# Hzone003: 뚝섬  
# Hzone001 광나루     
# Hzone008: 양화  
# Hzone007: 여의도    
# Hzone005: 반포  
# Hzone004: 잠원  
# Hzone002: 잠실  

In [ ]:
# 주차장일별['code'] 
codelist = []
for code in df['공원 코드']:
    codelist.append(code.replace('Hzone',""))

In [ ]:
df['code'] = codelist

In [ ]:
df['code']= pd.to_numeric(df['code'])

In [ ]:
# df.iloc[:,]
df_merge['날짜']= pd.to_datetime(df_merge['날짜'])

In [ ]:
df

In [ ]:
df['일반이용자(아침)'].replace(0,np.nan, inplace=True)
df['일반이용자(저녁)'].replace(0,np.nan,inplace=True)
df['일반이용자(낮)'].replace(0,np.nan,inplace=True)


In [ ]:
df.info()

In [ ]:
df.rename(columns={'현황 일시' : '날짜'},inplace=True)

In [ ]:
df['년도'] = df['날짜'].dt.year
df['월'] = df['날짜'].dt.month

In [ ]:
df

In [ ]:
nanmean= pd.DataFrame({'아침 평균':df.groupby(['code','월'])['일반이용자(아침)'].mean(),
'낮 평균':df.groupby(['code','월'])['일반이용자(낮)'].mean(),
'저녁 평균':df.groupby(['code','월'])['일반이용자(저녁)'].mean()},
)

In [ ]:
nanmean.reset_index(inplace=True)

In [ ]:
nanmean['합계'] = nanmean['아침 평균'] + nanmean['낮 평균'] + nanmean['저녁 평균']

In [ ]:
nanmean['아침 비율'] = nanmean['아침 평균']/nanmean['합계']
nanmean['저녁 비율'] = nanmean['저녁 평균']/nanmean['합계']
nanmean['낮 비율'] = nanmean['낮 평균']/nanmean['합계']

In [ ]:
nanmean.drop('합계', axis=1, inplace= True)

In [ ]:
nanmean[nanmean['code'] ==5]

In [ ]:
df = pd.merge(df,nanmean, on=['code', '월'])


In [ ]:
df['월'].unique()

In [ ]:
df['일반이용자(아침)'].fillna(df['아침 평균'], inplace=True)
df['일반이용자(낮)'].fillna(df['낮 평균'], inplace=True)
df['일반이용자(저녁)'].fillna(df['저녁 평균'], inplace=True)


In [ ]:
df

In [ ]:
df

In [ ]:
df.drop(columns=['날짜','공원명','공원 코드','아침 평균','낮 평균','저녁 평균'],axis=1,inplace=True)

--------

In [ ]:
df_merge3= pd.merge(df_merge, df, how='left', left_on=['code','월','년도'], right_on=['code','월','년도'])

In [ ]:
df_merge3['휴일여부']= df_merge3['휴일여부'].map({'주말' : 1, '평일' : 0, '휴일' : 1})

In [ ]:
df_merge3.drop(['최저기온 시각(hhmi)','최고기온 시각(hhmi)','평균 이슬점온도(°C)','평균 상대습도(%)'], axis=1, inplace= True)

In [ ]:
result  = []
for i in df_merge3['일강수량(mm)']:
    if i == 0 :
        result.append(0)
    else:
        result.append(1)

In [ ]:
df_merge3['비'] = result

In [ ]:
df_merge3

In [ ]:
code_month_mean=pd.DataFrame({'지점월별아침평균':df_merge3.groupby(['code','월','휴일여부'])['일반이용자(아침)'].mean(),'지점월별낮평균':df_merge3.groupby(['code','월','휴일여부'])['일반이용자(낮)'].mean(),'지점월별저녁평균':df_merge3.groupby(['code','월','휴일여부'])['일반이용자(저녁)'].mean()})
code_month_mean.reset_index(inplace=True)

In [ ]:
code_month_mean

In [ ]:
df_merge3=pd.merge(df_merge3,code_month_mean, on=['code','월', '휴일여부'])

In [ ]:
df_merge3['아침 비율'].fillna(df_merge3['지점월별아침평균']/(df_merge3.iloc[:,-3:].sum(axis=1)), inplace=True)
df_merge3['낮 비율'].fillna(df_merge3['지점월별낮평균']/(df_merge3.iloc[:,-3:].sum(axis=1)), inplace=True)
df_merge3['저녁 비율'].fillna(df_merge3['지점월별저녁평균']/(df_merge3.iloc[:,-3:].sum(axis=1)), inplace=True)

In [ ]:
df_merge3['일반이용자(아침)'].fillna(df_merge3['지점월별아침평균'],inplace=True)
df_merge3['일반이용자(낮)'].fillna(df_merge3['지점월별낮평균'],inplace=True)
df_merge3['일반이용자(저녁)'].fillna(df_merge3['지점월별저녁평균'],inplace=True)

------------
-------------

In [ ]:
# 파일 저장
# df_merge3.to_csv('../../Data/version1.csv',index=False)

code = 지구별에서 숫자값만 추출
지구별
PLT-001: 강서   
PLT-002: 난지   
PLT-003: 망원   
PLT-004: 이촌   
PLT-005: 뚝섬       
PLT-006: 광나루     
PLT-007: 양화   
PLT-008: 여의도     
PLT-009: 반포   
PLT-0010: 잠원  
PLT-0011: 잠실  

In [ ]:
df = pd.read_csv('../../Data/version1.csv')
df.head()

In [ ]:
df

In [ ]:
# df.drop('일',axis=1, inplace= True)

In [ ]:
df.head()

In [ ]:

# 상관계수 계산
correlation_matrix = df.iloc[:,1:].corr()

# 그림 크기 설정
plt.figure(figsize=(10, 8))

# heatmap 생성
sns.heatmap(correlation_matrix, 
            annot=True,  # 각 셀에 숫자 표시
            cmap='coolwarm',  # 색상 맵 설정
            vmin=-1, vmax=1,  # 색상 범위 설정
            center=0,  # 0을 중심으로 색상 대칭
            square=True,  # 정사각형 셀
            fmt='.2f')  # 소수점 둘째 자리까지 표시

# 제목 설정
plt.title('Correlation Heatmap', fontsize=16)

# 그래프 표시
plt.show()


In [ ]:
# df['아침이용비율'] =
df['아침이용비율']= df['일반이용자(아침)'] / (df['일반이용자(아침)']+df['일반이용자(낮)']+df['일반이용자(저녁)'])
df['낮이용비율']= df['일반이용자(낮)'] / (df['일반이용자(아침)']+df['일반이용자(낮)']+df['일반이용자(저녁)'])
df['저녁이용비율']= df['일반이용자(저녁)'] / (df['일반이용자(아침)']+df['일반이용자(낮)']+df['일반이용자(저녁)'])
# df

In [ ]:
# df['아침주차']= df['주차대수'] * df['아침이용비율']
# df['낮주차']= df['주차대수'] * df['낮이용비율']
# df['저녁주차']= df['주차대수'] * df['저녁이용비율']
# df.head()

In [ ]:
park_info=pd.read_csv('../../Data/한강공원 주차장 정보.csv',encoding= 'euc-kr')

In [ ]:
park_info.지구별.unique()

In [ ]:
park_info.head()

In [ ]:
park_info.head()

In [ ]:
df.head()

In [ ]:
df=pd.merge(df, park_info.loc[:,['주차장별','면수']], how='inner', left_on='주차장명', right_on='주차장별')

In [ ]:
df.drop('주차장별',axis=1, inplace= True)

In [ ]:
# df['아침주차비율'] = df['면수']*df['아침이용비율'] 
# df['낮주차비율'] = df['면수']*df['낮이용비율']
# df['저녁주차비율'] = df['면수']*df['저녁이용비율']
# df.head()

In [ ]:

# 그림 크기 설정
plt.figure(figsize=(10, 8))

sns.heatmap(
            df.loc[:,['주차대수','낮이용비율','아침이용비율','저녁이용비율']].corr(),
            annot=True,  # 각 셀에 숫자 표시
            cmap='coolwarm',  # 색상 맵 설정
            vmin=-1, vmax=1,  # 색상 범위 설정
            center=0,  # 0을 중심으로 색상 대칭
            square=True,  # 정사각형 셀
            fmt='.2f')  # 소수점 둘째 자리까지 표시

# 제목 설정
plt.title('Correlation Heatmap', fontsize=16)

# 그래프 표시
plt.show()


In [ ]:
df.head()

In [ ]:
grouped_df = df.groupby(['code', '월','휴일여부'], as_index=False)['주차대수'].sum()
grouped_df

In [ ]:
test=pd.merge(grouped_df, df.loc[:,['주차대수','code','월','주차장명','휴일여부']], how='right', left_on=['code','월','휴일여부'], right_on=['code','월','휴일여부'])

In [ ]:
test

In [ ]:
test['주차비율'] = (test['주차대수_y']/test['주차대수_x'])

In [ ]:
test

In [ ]:
df['주차비율'] = test['주차비율']

In [ ]:
df.rename(columns={'아침이용비율' : '월별아침이용자비율', '낮이용비율' : '월별낮이용자비율', '저녁이용비율' : '월별저녁이용자비율'},inplace= True)

In [ ]:
# df['주차대수']*df['월별아침이용자비율']
# df['일반이용자(아침)']*df['주차비율']
df['아침이용자']=df['주차비율'] * df['일반이용자(아침)']
df['낮이용자']=df['주차비율'] * df['일반이용자(낮)']
df['저녁이용자']=df['주차비율'] * df['일반이용자(저녁)']

In [ ]:
df.drop(['일반이용자(아침)','일반이용자(낮)','일반이용자(저녁)'],axis=1,inplace= True)

In [ ]:
# df.drop(['월별아침이용자비율','월별낮이용자비율','월별저녁이용자비율'],axis=1, inplace= True)

> 주차비율 = 각 주차장 주차대수 / 월별code 주차대수 합

In [ ]:
df['아침주차']=df['주차대수'] * df['월별아침이용자비율']
df['낮주차']=df['주차대수'] * df['월별낮이용자비율']
df['저녁주차']=df['주차대수'] * df['월별저녁이용자비율']


>> 시간대별 이용자 구분을 target값이 들어간 계산값으로 나눴기 때문에, 잘못됨

In [ ]:
df.drop(['아침주차','낮주차','저녁주차'],axis=1, inplace= True)

------------------------

In [ ]:
df=pd.read_csv('../../Data/서울 한강 일반 이용자 전처리본.csv',index_col=0)

In [ ]:
subway=pd.read_csv('../../Data/서울교통공사_역별 일별 시간대별 승하차인원(24.2~24.5).csv',encoding='euc-kr', index_col=0)

In [ ]:
df.drop(['일반이용자 합계','일반이용자(아침)','일반이용자(낮)','일반이용자(저녁)'], axis=1, inplace= True)